## Notebook for training GPR on DC dataset and generating set of suggestions

**Note: Dataset do not contain any NMC data**

## Batch: 4

In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs, PandasTools, Fragments, rdMolDescriptors, Descriptors, rdDepictor
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem.PandasTools import ChangeMoleculeRendering
from rdkit.Chem.MolStandardize.rdMolStandardize import LargestFragmentChooser
# Silence non-critical RDKit warnings to minimize unnecessary outputs
from rdkit import RDLogger
lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
## import train_test_split from sklearn
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process.kernels import RBF, ExpSineSquared, RationalQuadratic, WhiteKernel, Matern, ConstantKernel, DotProduct, PairwiseKernel 
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.stats import norm
from scipy.optimize import minimize
from scipy.special import erf
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%%bash
pwd
ls -ltr

/Users/riteshk/Library/CloudStorage/Box-Box/Research-postdoc/AD-AFB/Nat-Comm-R2/active-learning_wo_nmc_data
total 18624
-rw-r--r--@ 1 riteshk  staff  1485993 Apr 17 21:00 active_learning_batch_5.ipynb
-rw-r--r--@ 1 riteshk  staff  1474650 Apr 17 21:00 active_learning_batch_6.ipynb
-rw-r--r--@ 1 riteshk  staff  1381340 Apr 17 21:00 active_learning_batch_7.ipynb
-rw-r--r--@ 1 riteshk  staff   964972 Apr 17 21:42 active_learning_batch_1.ipynb
-rw-r--r--@ 1 riteshk  staff   984020 Apr 22 09:52 active_learning_batch_2.ipynb
-rw-r--r--@ 1 riteshk  staff  1113816 Apr 22 10:29 active_learning_batch_3.ipynb
-rw-r--r--@ 1 riteshk  staff  2113667 Apr 22 10:48 active_learning_batch_4.ipynb


### Reading & standardizing datasets

In [9]:
rem_till_b3 = [33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 56, 57, 60, 61, 62, 66, 67, 68, 70, 73, 74, 75, 76, 77, 78, 79, 104, 110, 111, 112, 113, 117, 118, 119]
rem_till_b3 += [121, 123, 127, 128, 129]
len(rem_till_b3)

40

In [ ]:
df = pd.read_csv('../../datasets/batch-4/label_data_post_batch3.csv') 
df.drop(index=rem_till_b3, inplace=True)
df.reset_index(inplace=True, drop=True)
df

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,norm_capacity_14,norm_capacity_15,norm_capacity_16,norm_capacity_17,norm_capacity_18,norm_capacity_19,norm_capacity_20,norm_capacity_21,norm_capacity_22,norm_capacity_23
0,COCCOC,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],-0.845301,-0.995151,1.062720,-0.357552,-0.308720,0.309456,0.693325,0.613072,...,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0
1,COCCOC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],-1.183255,-0.948338,0.615257,-0.582269,-1.010187,-0.522075,0.496896,0.301582,...,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0
2,COCCOCC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],-1.240814,-0.970769,0.671105,-0.607789,-1.124651,-0.436617,0.628824,0.421934,...,0.022233,0.02168,0.022967,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0
3,COCCOCC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],-1.240814,-0.970769,0.671105,-0.607789,-1.124651,-0.436617,0.628824,0.421934,...,0.000573,0.00046,0.000360,0.000407,0.0006,0.000593,0.000447,0.000333,0.0,0.0
4,CCOCCOC(C)(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],-0.879184,-1.539457,0.507075,-0.111830,0.800175,0.133053,0.926130,-0.208395,...,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,CCCCOCCOCCOCCOC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.067892,-1.253529,0.602357,0.775134,-0.714364,1.295648,0.646532,0.057886,...,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0
87,CCOC(=O)CN(C)CCOC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.618370,0.619338,0.080507,0.000244,0.093421,0.996731,0.388905,0.901693,...,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0
88,COCCN(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.910508,-0.897028,-0.304000,-0.934742,-0.208599,0.859589,0.955882,1.288009,...,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0
89,COCCN(C)C,[Li+].O=C1O[B-](F)(F)OC1=O,-0.910508,-0.897028,-0.304000,-0.934742,-0.208599,0.859589,0.955882,1.288009,...,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0


In [11]:
X = df.iloc[:,2:27] ## PCA-reduced solvent & salt descriptors and other non-molecular features
y = df['norm_capacity_3'] # normalized discharge capacity at 20th cycle (target variable)
std_scale = StandardScaler().fit(X)
X_std = std_scale.transform(X)
X_std = pd.DataFrame(X_std, columns=X.columns)

### Active learning workflow

#### Choose best hyperparameters for each kernel

In [12]:
def negative_log_likelihood_rbf(params):
    noise_level, length_scale, alpha = params
    kernel = RBF(length_scale=length_scale)
    white_kernel = WhiteKernel(noise_level=noise_level)
    composite_kernel = kernel + white_kernel
    gpr = GaussianProcessRegressor(kernel=composite_kernel, alpha=alpha, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42) 
    gpr.fit(X_std, y)
    pred_mean, pred_std = gpr.predict(X_std, return_std=True)
    log_likelihood = np.sum(norm.logpdf(y, loc=pred_mean, scale=pred_std))
    return -log_likelihood

def negative_log_likelihood_rq(params):
    noise_level, length_scale, alpha_k, alpha = params 
    kernel = RationalQuadratic(length_scale=length_scale, alpha=alpha_k)
    white_kernel = WhiteKernel(noise_level=noise_level)
    composite_kernel = kernel + white_kernel
    gpr = GaussianProcessRegressor(kernel=composite_kernel, alpha=alpha, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42)
    gpr.fit(X_std, y)
    pred_mean, pred_std = gpr.predict(X_std, return_std=True)
    log_likelihood = np.sum(norm.logpdf(y, loc=pred_mean, scale=pred_std))
    return -log_likelihood

def negative_log_likelihood_rbf_expsin(params):
    noise_level, length_scale, periodicity, alpha = params 
    kernel = RBF(length_scale=length_scale) + ExpSineSquared(length_scale=length_scale, periodicity=periodicity)
    white_kernel = WhiteKernel(noise_level=noise_level)
    composite_kernel = kernel + white_kernel
    gpr = GaussianProcessRegressor(kernel=composite_kernel, alpha=alpha, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42) 
    gpr.fit(X_std, y)
    pred_mean, pred_std = gpr.predict(X_std, return_std=True)
    log_likelihood = np.sum(norm.logpdf(y, loc=pred_mean, scale=pred_std))
    return -log_likelihood

def negative_log_likelihood_matern(params):
    noise_level, length_scale, alpha = params
    kernel = Matern(length_scale=length_scale, nu=1.5)
    white_kernel = WhiteKernel(noise_level=noise_level)
    composite_kernel = kernel + white_kernel
    gpr = GaussianProcessRegressor(kernel=composite_kernel, alpha=alpha, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42)
    gpr.fit(X_std, y)
    pred_mean, pred_std = gpr.predict(X_std, return_std=True)
    log_likelihood = np.sum(norm.logpdf(y, loc=pred_mean, scale=pred_std))
    return -log_likelihood

def negative_log_likelihood_pairwise(params):
    noise_level, length_scale, alpha = params
    kernel = PairwiseKernel(metric="polynomial")
    white_kernel = WhiteKernel(noise_level=noise_level)
    composite_kernel = kernel + white_kernel
    gpr = GaussianProcessRegressor(kernel=composite_kernel, alpha=alpha, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42)
    gpr.fit(X_std, y)
    pred_mean, pred_std = gpr.predict(X_std, return_std=True)
    log_likelihood = np.sum(norm.logpdf(y, loc=pred_mean, scale=pred_std))
    return -log_likelihood

##### Pairwise kernel

In [13]:
initial_guess = [0.15, 0.01, 0.02] # initial guess for noise_level, length_scale, alpha
param_bounds = [(1e-4, 1.0), (1e-5, 50.0), (1e-4, 0.1)] # bounds for noise_level, length_scale, alpha
result = minimize(negative_log_likelihood_pairwise, initial_guess, bounds=param_bounds)
optimized_hyperparameters = result.x
optimized_noise_level, optimized_length_scale, optimized_alpha = optimized_hyperparameters
print("Optimized noise_level:", optimized_noise_level)
print("Optimized length_scale:", optimized_length_scale)
print("Optimized alpha:", optimized_alpha)

/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specif

Optimized noise_level: 0.14999915148797846
Optimized length_scale: 0.01
Optimized alpha: 0.00896866890148441


/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


##### RationalQuadratic kernel

In [14]:
initial_guess = [0.15, 0.01, 0.01, 0.02] # initial guess for noise_level, length_scale, alpha_k, alpha
param_bounds = [(1e-4, 1.0), (1e-5, 50.0), (1e-5, 50.0), (1e-4, 0.1)] # bounds for noise_level, length_scale, alpha_k, alpha
result = minimize(negative_log_likelihood_rq, initial_guess, bounds=param_bounds)
optimized_hyperparameters = result.x
optimized_noise_level, optimized_length_scale, optimized_alpha_k, optimized_alpha = optimized_hyperparameters
print("Optimized noise_level:", optimized_noise_level)
print("Optimized length_scale:", optimized_length_scale)
print("Optimized alpha_k:", optimized_alpha_k)
print("Optimized alpha:", optimized_alpha)

/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warning

Optimized noise_level: 0.43349265296972167
Optimized length_scale: 16.682873603261697
Optimized alpha_k: 0.01
Optimized alpha: 0.0057964748939704


##### Matern-3/2 kernel

In [15]:
initial_guess = [0.15, 0.01, 0.02] # initial guess for noise_level, length_scale, alpha
param_bounds = [(1e-4, 1.0), (1e-5, 50.0), (1e-4, 0.1)] # bounds for noise_level, length_scale, alpha
result = minimize(negative_log_likelihood_matern, initial_guess, bounds=param_bounds)
optimized_hyperparameters = result.x
optimized_noise_level, optimized_length_scale, optimized_alpha = optimized_hyperparameters
print("Optimized noise_level:", optimized_noise_level)
print("Optimized length_scale:", optimized_length_scale)
print("Optimized alpha:", optimized_alpha)

Optimized noise_level: 0.15
Optimized length_scale: 0.01
Optimized alpha: 0.012060848751972523


##### RBF-ExpineSquared kernel

In [16]:
initial_guess = [0.15, 0.01, 1.0, 0.02] # initial guess for noise_level, length_scale, periodicity, alpha
param_bounds = [(1e-4, 1.0), (1e-5, 50.0), (1e-2, 10.0), (1e-4, 0.1)] # bounds for noise_level, length_scale, periodicity, alpha
result = minimize(negative_log_likelihood_rbf_expsin, initial_guess, bounds=param_bounds)
optimized_hyperparameters = result.x
optimized_noise_level, optimized_length_scale, optimized_periodicity, optimized_alpha = optimized_hyperparameters
print("Optimized noise_level:", optimized_noise_level)
print("Optimized length_scale:", optimized_length_scale)
print("Optimized periodicity:", optimized_periodicity)
print("Optimized alpha:", optimized_alpha)

/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__periodicity is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:429: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a 

Optimized noise_level: 0.15
Optimized length_scale: 0.01
Optimized periodicity: 1.0
Optimized alpha: 0.01999130779990663


#### Train surrogate models

Note: no need to run again, saved model checkpoints have been provided

In [ ]:
## change all hyperparameters accordingly
optimized_pairwise_kernel = PairwiseKernel(metric="polynomial") + WhiteKernel(noise_level=0.14999915148797846)
optimized_matern_kernel = Matern(length_scale=0.01, nu=1.5) + WhiteKernel(noise_level=0.15)
optimized_rbfexpsin_kernel = RBF(length_scale=0.01) + ExpSineSquared(length_scale=0.01, periodicity=1.0) + WhiteKernel(noise_level=0.15)
optimized_rq_kernel = RationalQuadratic(length_scale=16.682873603261697, alpha=0.01) + WhiteKernel(noise_level=0.43349265296972167)

gpr_models = [GaussianProcessRegressor(kernel=optimized_pairwise_kernel, alpha=0.00896866890148441, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42),
              GaussianProcessRegressor(kernel=optimized_matern_kernel, alpha=0.012060848751972523, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42),
              GaussianProcessRegressor(kernel=optimized_rq_kernel, alpha=0.0057964748939704, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42),
              GaussianProcessRegressor(kernel=optimized_rbfexpsin_kernel, alpha=0.01999130779990663, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42)]

model_names = ['../../models/batch-4/pairwise_batch4_wo_nmc_data.pkl', '../../models/batch-4/matern_batch4_wo_nmc_data.pkl', '../../models/batch-4/rq_batch4_wo_nmc_data.pkl', '../../models/batch-4/rbf-ess_batch4_wo_nmc_data.pkl']
k = 0
for model in gpr_models:
    print("fitting model: ", k)
    model.fit(X_std, y)
    pickle.dump(model, open(model_names[k], 'wb'))
    k += 1

fitting model:  0
fitting model:  1
fitting model:  2
fitting model:  3


#### BMA: aggregate predictions

##### Acquisition function (Expected improvement)

In [18]:
## final corrected & verified one to be used
def calc_EI(y_pred, y_pred_un, y_pred_un_uncer, epsilon=0.01):
    y_best = np.max(y_pred)
    EI = []
    explore = []
    exploit = []

    for i in range(len(y_pred_un)):
        if y_pred_un_uncer[i] != 0:
            
            # Calculate the cumulative distribution function (CDF) for the Gaussian distribution
            z = (y_pred_un[i] - y_best - epsilon) / y_pred_un_uncer[i]
            # z = (y_pred_un[i] - y_best - epsilon) / y_pred_un_uncer[i]
            cdf_z = 0.5 * (1 + erf(z / np.sqrt(2)))
            pdf_z = np.exp(-0.5 * z**2) / np.sqrt(2 * np.pi)

            # Calculate Expected Improvement
            expected_improvement = y_pred_un_uncer[i] * (z * cdf_z) + y_pred_un_uncer[i] * pdf_z
            exploitation = y_pred_un_uncer[i] * z * cdf_z
            exploration = y_pred_un_uncer[i] * pdf_z
            EI.append(expected_improvement)
            explore.append(exploration)
            exploit.append(exploitation)
        else:
            EI.append(0.0)
    return EI, exploit, explore

In [ ]:
## virtual search space for batch-4 (electrolytes containing solvent combinations tested in batch-3 removed)
path = '/Users/riteshk/Library/CloudStorage/Box-Box/Research-postdoc/AD-AFB/data-codes-sharing/datasets'
df_unlabel = pd.read_csv(f'../../datasets/batch-4/virtual_search_space_for_batch4.csv')
df_unlabel

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,salt_ecfp_pca_5,salt_ecfp_pca_6,salt_ecfp_pca_7,salt_ecfp_pca_8,salt_ecfp_pca_9,mol_wt_solv,mol_wt_salt,conc_salt_1,theor_capacity,amt_electrolyte
0,CN(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.325380,-0.919052,-0.279367,-0.373535,0.936724,-0.161937,-0.377185,0.259444,...,0.128733,-0.322339,0.258767,0.301779,-0.272648,73.052764,186.939685,1,150,50
1,CN1CCN(C)C1=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.013958,-0.868963,0.700464,0.728341,0.681048,-1.181697,0.018457,0.793475,...,0.128733,-0.322339,0.258767,0.301779,-0.272648,114.079313,186.939685,1,150,50
2,CN(C)C(=O)N(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.571448,0.079720,-0.117841,-0.342583,1.301206,-0.264839,-0.516489,0.217482,...,0.128733,-0.322339,0.258767,0.301779,-0.272648,116.094963,186.939685,1,150,50
3,CB(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.152976,-1.321495,0.631553,-0.113769,0.728616,-0.486733,-0.311699,0.151570,...,0.128733,-0.322339,0.258767,0.301779,-0.272648,70.058995,186.939685,1,150,50
4,[CH2]N(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.283297,-0.876057,-0.318283,-0.547668,0.955202,-0.313588,-0.356929,0.299023,...,0.128733,-0.322339,0.258767,0.301779,-0.272648,72.044939,186.939685,1,150,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999909,CC1ON(C)C(C)C1S(C)(=O)=O,[Li+].O=C1O[B-](F)(F)OC1=O,0.771012,-1.078565,0.565077,0.216605,0.613562,-0.742397,-0.257837,0.243970,...,-0.387958,-0.076666,0.008496,0.161338,0.109063,193.077264,144.001775,1,150,50
999910,COC(=O)C1(N2CCCN(C)C2=O)CCCC1,[Li+].O=C1O[B-](F)(F)OC1=O,0.881747,0.625623,1.808971,0.306581,0.123581,-0.469934,-0.451538,1.337094,...,-0.387958,-0.076666,0.008496,0.161338,0.109063,240.147392,144.001775,1,150,50
999911,COC(=O)N1CCC(OS(C)(=O)=O)CC1=O,[Li+].O=C1O[B-](F)(F)OC1=O,0.700964,0.588889,1.459777,0.806459,-0.042994,-0.081386,0.196396,-0.195416,...,-0.387958,-0.076666,0.008496,0.161338,0.109063,251.046358,144.001775,1,150,50
999912,CN(C)C(=O)CN(C)C(=O)C1CCCC1(F)F,[Li+].O=C1O[B-](F)(F)OC1=O,0.558235,0.889818,-0.240008,-0.734436,0.749283,0.799906,-0.856418,0.214675,...,-0.387958,-0.076666,0.008496,0.161338,0.109063,248.133634,144.001775,1,150,50


In [21]:
X_un = df_unlabel.iloc[:,2:]
X_un_std = std_scale.transform(X_un)
X_un_std = pd.DataFrame(X_un_std, columns=X_un.columns)

##### Calculate model weights & obtained aggregated mean ($\mu^{aggr}$), uncertainty ($\sigma^{aggr}$), & EI ($EI^{aggr}$)

In [ ]:
# Calculate Model Weights using BMA (first order)
model_names = ['../../models/batch-4/pairwise_batch4_wo_nmc_data.pkl', '../../models/batch-4/matern_batch4_wo_nmc_data.pkl', '../../models/batch-4/rq_batch4_wo_nmc_data.pkl', '../../models/batch-4/rbf-ess_batch4_wo_nmc_data.pkl']
model_weights = []
uncertainties = []
predictions = []
y_label_preds = []
for model in model_names:
    gpr = pickle.load(open(model, 'rb'))
    y_un = gpr.predict(X_un_std)
    predictions.append(y_un)
    individual_uncertainties = gpr.predict(X_un_std, return_std=True)[1]
    uncertainties.append(individual_uncertainties)
    likelihoods = norm.pdf(y_un, loc=gpr.predict(X_un_std), scale=individual_uncertainties)
    prior_beliefs = 1.0  # Non-informative prior
    posterior = likelihoods * prior_beliefs
    model_weights.append(posterior / np.sum(posterior))
    y_ = gpr.predict(X_std)
    y_label_preds.append(y_)

In [23]:
df_unlabel['uncertainty_1'] = uncertainties[0]; df_unlabel['uncertainty_2'] = uncertainties[1]; df_unlabel['uncertainty_3'] = uncertainties[2]; df_unlabel['uncertainty_4'] = uncertainties[3]
df_unlabel['prediction_1'] = predictions[0]; df_unlabel['prediction_2'] = predictions[1]; df_unlabel['prediction_3'] = predictions[2]; df_unlabel['prediction_4'] = predictions[3]
df_unlabel['explore_1'] = calc_EI(y_label_preds[0], predictions[0], uncertainties[0])[2]; df_unlabel['exploit_1'] = calc_EI(y_label_preds[0], predictions[0], uncertainties[0])[1]
df_unlabel['explore_2'] = calc_EI(y_label_preds[1], predictions[1], uncertainties[1])[2]; df_unlabel['exploit_2'] = calc_EI(y_label_preds[1], predictions[1], uncertainties[1])[1]
df_unlabel['explore_3'] = calc_EI(y_label_preds[2], predictions[2], uncertainties[2])[2]; df_unlabel['exploit_3'] = calc_EI(y_label_preds[2], predictions[2], uncertainties[2])[1]
df_unlabel['explore_4'] = calc_EI(y_label_preds[3], predictions[3], uncertainties[3])[2]; df_unlabel['exploit_4'] = calc_EI(y_label_preds[3], predictions[3], uncertainties[3])[1]
df_unlabel['EI_1'] = calc_EI(y_label_preds[0], predictions[0], uncertainties[0])[0]; df_unlabel['EI_2'] = calc_EI(y_label_preds[1], predictions[1], uncertainties[1])[0]; df_unlabel['EI_3'] = calc_EI(y_label_preds[2], predictions[2], uncertainties[2])[0]; df_unlabel['EI_4'] = calc_EI(y_label_preds[3], predictions[3], uncertainties[3])[0]
df_unlabel

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,explore_2,exploit_2,explore_3,exploit_3,explore_4,exploit_4,EI_1,EI_2,EI_3,EI_4
0,CN(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.325380,-0.919052,-0.279367,-0.373535,0.936724,-0.161937,-0.377185,0.259444,...,0.001504,-0.001364,0.008947,-0.007694,0.000811,-0.000743,0.000277,0.000141,0.001253,0.000067
1,CN1CCN(C)C1=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.013958,-0.868963,0.700464,0.728341,0.681048,-1.181697,0.018457,0.793475,...,0.005108,-0.004497,0.005870,-0.005149,0.004218,-0.003747,0.001317,0.000612,0.000721,0.000471
2,CN(C)C(=O)N(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.571448,0.079720,-0.117841,-0.342583,1.301206,-0.264839,-0.516489,0.217482,...,0.001291,-0.001176,0.005548,-0.004878,0.001001,-0.000917,0.000239,0.000115,0.000670,0.000084
3,CB(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.152976,-1.321495,0.631553,-0.113769,0.728616,-0.486733,-0.311699,0.151570,...,0.004544,-0.003996,0.010697,-0.009082,0.001728,-0.001559,0.001277,0.000548,0.001615,0.000169
4,[CH2]N(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.283297,-0.876057,-0.318283,-0.547668,0.955202,-0.313588,-0.356929,0.299023,...,0.001411,-0.001281,0.008244,-0.007123,0.000889,-0.000814,0.000244,0.000130,0.001121,0.000075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999909,CC1ON(C)C(C)C1S(C)(=O)=O,[Li+].O=C1O[B-](F)(F)OC1=O,0.771012,-1.078565,0.565077,0.216605,0.613562,-0.742397,-0.257837,0.243970,...,0.001442,-0.001313,0.003483,-0.003110,0.003287,-0.002957,0.000150,0.000129,0.000373,0.000330
999910,COC(=O)C1(N2CCCN(C)C2=O)CCCC1,[Li+].O=C1O[B-](F)(F)OC1=O,0.881747,0.625623,1.808971,0.306581,0.123581,-0.469934,-0.451538,1.337094,...,0.017417,-0.014746,0.007169,-0.006274,0.065044,-0.050549,0.001064,0.002672,0.000895,0.014495
999911,COC(=O)N1CCC(OS(C)(=O)=O)CC1=O,[Li+].O=C1O[B-](F)(F)OC1=O,0.700964,0.588889,1.459777,0.806459,-0.042994,-0.081386,0.196396,-0.195416,...,0.014367,-0.012270,0.008101,-0.007053,0.038147,-0.031283,0.001432,0.002097,0.001048,0.006864
999912,CN(C)C(=O)CN(C)C(=O)C1CCCC1(F)F,[Li+].O=C1O[B-](F)(F)OC1=O,0.558235,0.889818,-0.240008,-0.734436,0.749283,0.799906,-0.856418,0.214675,...,0.004062,-0.003632,0.006151,-0.005410,0.026148,-0.022007,0.000047,0.000429,0.000741,0.004141


In [24]:
def calc_aggr_uncer(uncer_1, w_1, pred_1, uncer_2, w_2, pred_2, uncer_3, w_3, pred_3, uncer_4, w_4, pred_4):
    uncer = [uncer_1, uncer_2, uncer_3, uncer_4]
    pred = [pred_1, pred_2, pred_3, pred_4]
    weight = [w_1, w_2, w_3, w_4]
    pred_aggr = w_1 * pred_1 + w_2 * pred_2 + w_3 * pred_3 + w_4 * pred_4
    sum = 0
    for i in range(4):
        sum += weight[i] * (uncer[i]**2 + (pred[i] - pred_aggr)**2)
    aggr_uncer = np.sqrt(sum)
    return aggr_uncer

In [25]:
df_unlabel['prediction_aggr'] = df_unlabel['prediction_1'] * model_weights[0] + df_unlabel['prediction_2'] * model_weights[1] + df_unlabel['prediction_3'] * model_weights[2] + df_unlabel['prediction_4'] * model_weights[3]
df_unlabel['uncertainty_aggr'] = calc_aggr_uncer(df_unlabel['uncertainty_1'], model_weights[0], df_unlabel['prediction_1'], df_unlabel['uncertainty_2'], model_weights[1], df_unlabel['prediction_2'], df_unlabel['uncertainty_3'], model_weights[2], df_unlabel['prediction_3'], df_unlabel['uncertainty_4'], model_weights[3], df_unlabel['prediction_4'])
df_unlabel['explore_aggr'] = df_unlabel['explore_1'] * model_weights[0] + df_unlabel['explore_2'] * model_weights[1] + df_unlabel['explore_3'] * model_weights[2] + df_unlabel['explore_4'] * model_weights[3]
df_unlabel['exploit_aggr'] = df_unlabel['exploit_1'] * model_weights[0] + df_unlabel['exploit_2'] * model_weights[1] + df_unlabel['exploit_3'] * model_weights[2] + df_unlabel['exploit_4'] * model_weights[3]
df_unlabel['ratio_aggr'] = df_unlabel['exploit_aggr'] / df_unlabel['explore_aggr']

## 'EI_aggr' is the final rank by which candidate electrolytes are selected for experimental validation
df_unlabel['EI_aggr'] = df_unlabel['EI_1'] * model_weights[0] + df_unlabel['EI_2'] * model_weights[1] + df_unlabel['EI_3'] * model_weights[2] + df_unlabel['EI_4'] * model_weights[3]
df_unlabel

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,EI_1,EI_2,EI_3,EI_4,prediction_aggr,uncertainty_aggr,explore_aggr,exploit_aggr,ratio_aggr,EI_aggr
0,CN(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.325380,-0.919052,-0.279367,-0.373535,0.936724,-0.161937,-0.377185,0.259444,...,0.000277,0.000141,0.001253,0.000067,4.920505e-07,0.000567,1.577585e-08,-1.382427e-08,-0.876293,1.951582e-09
1,CN1CCN(C)C1=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.013958,-0.868963,0.700464,0.728341,0.681048,-1.181697,0.018457,0.793475,...,0.001317,0.000612,0.000721,0.000471,5.309894e-07,0.000587,2.676442e-08,-2.336907e-08,-0.873139,3.395357e-09
2,CN(C)C(=O)N(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.571448,0.079720,-0.117841,-0.342583,1.301206,-0.264839,-0.516489,0.217482,...,0.000239,0.000115,0.000670,0.000084,8.201231e-08,0.000545,1.090491e-08,-9.729020e-09,-0.892169,1.175889e-09
3,CB(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.152976,-1.321495,0.631553,-0.113769,0.728616,-0.486733,-0.311699,0.151570,...,0.001277,0.000548,0.001615,0.000169,1.124436e-06,0.000696,3.133238e-08,-2.701137e-08,-0.862091,4.321005e-09
4,[CH2]N(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.283297,-0.876057,-0.318283,-0.547668,0.955202,-0.313588,-0.356929,0.299023,...,0.000244,0.000130,0.001121,0.000075,3.800096e-07,0.000556,1.443452e-08,-1.269996e-08,-0.879833,1.734560e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999909,CC1ON(C)C(C)C1S(C)(=O)=O,[Li+].O=C1O[B-](F)(F)OC1=O,0.771012,-1.078565,0.565077,0.216605,0.613562,-0.742397,-0.257837,0.243970,...,0.000150,0.000129,0.000373,0.000330,-2.190059e-07,0.000587,9.735245e-09,-8.763117e-09,-0.900143,9.721282e-10
999910,COC(=O)C1(N2CCCN(C)C2=O)CCCC1,[Li+].O=C1O[B-](F)(F)OC1=O,0.881747,0.625623,1.808971,0.306581,0.123581,-0.469934,-0.451538,1.337094,...,0.001064,0.002672,0.000895,0.014495,2.218061e-08,0.000644,6.739281e-08,-5.461765e-08,-0.810437,1.277516e-08
999911,COC(=O)N1CCC(OS(C)(=O)=O)CC1=O,[Li+].O=C1O[B-](F)(F)OC1=O,0.700964,0.588889,1.459777,0.806459,-0.042994,-0.081386,0.196396,-0.195416,...,0.001432,0.002097,0.001048,0.006864,2.269506e-08,0.000644,5.312611e-08,-4.471643e-08,-0.841703,8.409688e-09
999912,CN(C)C(=O)CN(C)C(=O)C1CCCC1(F)F,[Li+].O=C1O[B-](F)(F)OC1=O,0.558235,0.889818,-0.240008,-0.734436,0.749283,0.799906,-0.856418,0.214675,...,0.000047,0.000429,0.000741,0.004141,-4.203621e-07,0.000694,2.723567e-08,-2.335844e-08,-0.857642,3.877223e-09


##### Save top 10000 predictions

In [26]:
df_unlabel_ = df_unlabel.copy()
df_unlabel_ = df_unlabel_.sort_values(by='EI_aggr', ascending=False)
df_unlabel_10000 = df_unlabel_.iloc[:10000,:]
df_unlabel_10000

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,EI_1,EI_2,EI_3,EI_4,prediction_aggr,uncertainty_aggr,explore_aggr,exploit_aggr,ratio_aggr,EI_aggr
66106,COCCOCCOC(F)(F)Cl,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.013246,-1.132293,1.203856,-0.549821,0.238278,0.290024,1.338129,0.446974,...,0.008311,0.010022,0.009974,0.004824,3.927265e-06,0.001535,2.032648e-07,-1.421704e-07,-0.699434,6.109439e-08
223368,COCCOCC(F)(OC)C(F)(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.020865,-1.124835,1.256503,-0.548366,0.183966,0.306774,1.383833,0.462789,...,0.008990,0.010922,0.009346,0.005423,3.647808e-06,0.001467,2.036904e-07,-1.445650e-07,-0.709729,5.912541e-08
193870,COCCOCCC(F)(F)C(F)(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.995461,-1.139384,1.208259,-0.557954,0.205390,0.322203,1.391092,0.433611,...,0.008560,0.010429,0.010905,0.005210,3.500622e-06,0.001431,2.022702e-07,-1.431818e-07,-0.707874,5.908839e-08
111656,COCCOCC(F)(F)OCF,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.979777,-1.126940,1.238547,-0.543210,0.172293,0.267553,1.378745,0.361646,...,0.008655,0.010482,0.012502,0.005346,3.262107e-06,0.001371,2.025798e-07,-1.438307e-07,-0.709995,5.874905e-08
255451,COCCOCCOCCOCCOCC(F)(F)CCCCC[Si](OC)(OC)OC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.309362,-1.134852,1.181017,-0.133862,0.039111,0.605024,1.403487,0.326658,...,0.027248,0.013353,0.001753,0.017335,1.273534e-06,0.000879,1.882993e-07,-1.301604e-07,-0.691242,5.813893e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252824,COCCC(=O)N1CCN(C(=O)C(C)(C)C)CC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.281381,0.522806,1.360425,0.402163,0.443373,-0.153257,1.562089,0.615370,...,0.006078,0.003086,0.000954,0.004148,7.580584e-07,0.000681,7.792661e-08,-6.365001e-08,-0.816794,1.427660e-08
327278,COC(=O)CC1C(=O)N(C)CCN1C(=O)N1CCC(C)(OC)C1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.904417,0.914748,1.975455,0.537320,-0.080118,-0.456326,-0.384179,1.347751,...,0.005910,0.003914,0.001474,0.008205,1.450842e-07,0.000674,7.929405e-08,-6.501750e-08,-0.819954,1.427654e-08
800157,COC(=O)C1CCCN(C=O)CN1C,[Li+].O=C1O[B-](F)(F)OC1=O,1.113584,1.066724,1.412924,0.457473,-0.573161,-0.069135,0.605337,-0.073147,...,0.001832,0.003478,0.001145,0.014404,1.267539e-07,0.000647,7.544169e-08,-6.116595e-08,-0.810771,1.427573e-08
295532,COCCOCCOCCCC(=O)C(C)(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.051868,-0.115286,1.282676,-0.037341,0.381647,0.783399,1.163550,0.014020,...,0.006641,0.003090,0.000989,0.001506,1.299337e-06,0.000789,7.584306e-08,-6.156749e-08,-0.811775,1.427557e-08


In [27]:
df_unlabel_uniq = df_unlabel_10000.drop_duplicates(subset=['solv_comb_sm'], keep='first') ## only keeping unique solvent combinations for selection purposes; these compounds were manually searched in emolecules to finf purchasable compounds
df_unlabel_uniq['solv_comb_sm'] = df_unlabel_uniq['solv_comb_sm'].apply(lambda x: Chem.MolToSmiles(Chem.MolFromSmiles(x)))
df_unlabel_uniq[['solv_comb_sm', 'salt_comb_sm', 'prediction_aggr', 'uncertainty_aggr', 'explore_aggr', 'exploit_aggr', 'ratio_aggr', 'EI_aggr']].to_csv('../datasets/batch-4/top_10000_suggestions_batch4_uniq_solvents_wo_nmc_data.csv', index=False)
df_unlabel_uniq

/var/folders/s5/lmr5dh0n4bsgnpxrzwlts5lw0000gn/T/ipykernel_33581/1804567059.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unlabel_uniq['solv_comb_sm'] = df_unlabel_uniq['solv_comb_sm'].apply(lambda x: Chem.MolToSmiles(Chem.MolFromSmiles(x)))


,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,EI_1,EI_2,EI_3,EI_4,prediction_aggr,uncertainty_aggr,explore_aggr,exploit_aggr,ratio_aggr,EI_aggr
66106,COCCOCCOC(F)(F)Cl,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.013246,-1.132293,1.203856,-0.549821,0.238278,0.290024,1.338129,0.446974,...,0.008311,0.010022,0.009974,0.004824,3.927265e-06,0.001535,2.032648e-07,-1.421704e-07,-0.699434,6.109439e-08
223368,COCCOCC(F)(OC)C(F)(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.020865,-1.124835,1.256503,-0.548366,0.183966,0.306774,1.383833,0.462789,...,0.008990,0.010922,0.009346,0.005423,3.647808e-06,0.001467,2.036904e-07,-1.445650e-07,-0.709729,5.912541e-08
193870,COCCOCCC(F)(F)C(F)(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.995461,-1.139384,1.208259,-0.557954,0.205390,0.322203,1.391092,0.433611,...,0.008560,0.010429,0.010905,0.005210,3.500622e-06,0.001431,2.022702e-07,-1.431818e-07,-0.707874,5.908839e-08
111656,COCCOCC(F)(F)OCF,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.979777,-1.126940,1.238547,-0.543210,0.172293,0.267553,1.378745,0.361646,...,0.008655,0.010482,0.012502,0.005346,3.262107e-06,0.001371,2.025798e-07,-1.438307e-07,-0.709995,5.874905e-08
255451,COCCOCCOCCOCCOCC(F)(F)CCCCC[Si](OC)(OC)OC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.309362,-1.134852,1.181017,-0.133862,0.039111,0.605024,1.403487,0.326658,...,0.027248,0.013353,0.001753,0.017335,1.273534e-06,0.000879,1.882993e-07,-1.301604e-07,-0.691242,5.813893e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,CCCOOC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.920924,-1.603036,0.550274,0.249128,-0.250719,0.486272,0.282188,0.202565,...,0.003560,0.001788,0.005234,0.000239,1.892553e-06,0.000933,7.407750e-08,-5.979962e-08,-0.807257,1.427788e-08
252824,COCCC(=O)N1CCN(C(=O)C(C)(C)C)CC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.281381,0.522806,1.360425,0.402163,0.443373,-0.153257,1.562089,0.615370,...,0.006078,0.003086,0.000954,0.004148,7.580584e-07,0.000681,7.792661e-08,-6.365001e-08,-0.816794,1.427660e-08
327278,COC(=O)CC1C(=O)N(C)CCN1C(=O)N1CCC(C)(OC)C1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.904417,0.914748,1.975455,0.537320,-0.080118,-0.456326,-0.384179,1.347751,...,0.005910,0.003914,0.001474,0.008205,1.450842e-07,0.000674,7.929405e-08,-6.501750e-08,-0.819954,1.427654e-08
295532,COCCOCCOCCCC(=O)C(C)(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.051868,-0.115286,1.282676,-0.037341,0.381647,0.783399,1.163550,0.014020,...,0.006641,0.003090,0.000989,0.001506,1.299337e-06,0.000789,7.584306e-08,-6.156749e-08,-0.811775,1.427557e-08


In [37]:
df_unlabel_uniq_ = pd.read_csv(f'{path}/batch-4/top_10000_suggestions_batch4_uniq_solvents.csv')
df_unlabel_uniq_

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,theor_capacity,amt_electrolyte,pressure_type,rank,prediction_aggr,uncertainty_aggr,explore_aggr,exploit_aggr,ratio_aggr,EI_aggr
0,CN1CC(=O)N(C)CC(=O)N(C)CC(=O)N(C)CC(=O)N(C)CC(...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.842454,-0.969284,0.843293,0.542591,0.659271,-1.028246,-0.227193,0.548413,...,150,50,2,854.0,3.809569e-07,0.000613,4.717666e-08,-3.970714e-08,-5.315888,7.469522e-09
1,CN1C(=O)N2CN3C(=O)N4CN5C(=O)N(C)C6C5N(CN5C(=O)...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.140354,-0.922768,0.777714,0.484251,0.590515,-0.894770,-0.217753,0.316862,...,150,50,2,3380.0,3.333645e-07,0.000585,3.230432e-08,-2.769543e-08,-6.009128,4.608893e-09
2,CN1CC(=O)N(C)CC(=O)N(C)CC(=O)N(C)CC(=O)N(C)CC(...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.842454,-0.969284,0.843293,0.542591,0.659271,-1.028246,-0.227193,0.548413,...,150,50,2,4319.0,4.195568e-07,0.000587,3.019479e-08,-2.597395e-08,-6.153732,4.220845e-09
3,CN1CCCC(=O)N(C)CC(=O)N(C)CC(=O)N(C)CC(=O)N(C)C...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.244749,-0.718715,0.816326,0.622103,0.842643,-1.134638,0.343202,0.456895,...,150,50,2,1275.0,4.778951e-07,0.000616,4.250074e-08,-3.603028e-08,-5.568422,6.470464e-09
4,CCCN1CC(=O)N(C)CC(=O)N(C)CC(=O)N(C)CC(=O)N(C)C...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.545357,-1.096163,0.435487,1.288385,0.482219,-0.659059,-0.329151,0.365688,...,150,50,2,506.0,3.220242e-07,0.000620,5.466365e-08,-4.557863e-08,-5.016902,9.085015e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8744,COCCOCCCCCCCCCCC[Si](OC)(OC)OC,[Li+].O=C1O[B-](F)(F)OC1=O,-0.908149,-0.944494,0.971822,0.043047,-0.735702,0.985443,0.804811,0.287560,...,150,50,2,7032.0,7.301459e-07,0.000644,2.668225e-08,-2.318253e-08,-6.624107,3.499721e-09
8745,CO[Si](OC)(OC)C([Si](OC)(OC)OC)([Si](OC)(OC)OC...,[Li+].O=C1O[B-](F)(F)OC1=O,-0.473794,-1.393888,1.066949,-0.607220,0.353716,-0.159430,0.584239,-0.017302,...,150,50,2,4871.0,3.959835e-07,0.000585,3.002041e-08,-2.597474e-08,-6.420383,4.045668e-09
8746,CCCCCCCCCCCCOCCOCCC[Si](OC)(OC)OC,[Li+].O=C1O[B-](F)(F)OC1=O,-1.076872,-1.681724,0.426816,1.007279,-0.740516,1.312715,0.261993,-0.277779,...,150,50,2,1628.0,5.599071e-07,0.000624,4.023875e-08,-3.430591e-08,-5.782378,5.932837e-09
8747,COCCOCCCOCCCCCCCCCCC[Si](OC)(OC)OC,[Li+].O=C1O[B-](F)(F)OC1=O,-0.915870,-0.938476,0.970829,0.044332,-0.738988,0.996804,0.802128,0.293352,...,150,50,2,1700.0,6.721882e-07,0.000651,3.951917e-08,-3.366613e-08,-5.751911,5.853034e-09


In [39]:
df_unlabel_uniq_['solv_comb_sm'] = df_unlabel_uniq_['solv_comb_sm'].apply(lambda x: Chem.MolToSmiles(Chem.MolFromSmiles(x)))
# df_sugg_comm = pd.merge(df_unlabel_uniq_, df_unlabel_uniq, on=['solv_comb_sm', 'salt_comb_sm'], how='inner') ## unique electrolytes
df_sugg_comm = pd.merge(df_unlabel_uniq_, df_unlabel_uniq, on=['solv_comb_sm'], how='inner') ## unique solvents
df_sugg_comm

,solv_comb_sm,salt_comb_sm_x,solv_ecfp_pca_0_x,solv_ecfp_pca_1_x,solv_ecfp_pca_2_x,solv_ecfp_pca_3_x,solv_ecfp_pca_4_x,solv_ecfp_pca_5_x,solv_ecfp_pca_6_x,solv_ecfp_pca_7_x,...,EI_1,EI_2,EI_3,EI_4,prediction_aggr_y,uncertainty_aggr_y,explore_aggr_y,exploit_aggr_y,ratio_aggr_y,EI_aggr_y
0,CN1CC(=O)N(C)CC(=O)N(C)CC(=O)N(C)CC(=O)N(C)CC(...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.842454,-0.969284,0.843293,0.542591,0.659271,-1.028246,-0.227193,0.548413,...,0.005391,0.007562,0.002048,0.025140,-2.614581e-08,0.000745,1.042037e-07,-8.254011e-08,-0.792104,2.166355e-08
1,CN1C(=O)N2CN3C(=O)N4CN5C(=O)N(C)C6C5N(CN5C(=O)...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.140354,-0.922768,0.777714,0.484251,0.590515,-0.894770,-0.217753,0.316862,...,0.003411,0.005134,0.001785,0.017879,-7.025629e-08,0.000729,8.499894e-08,-6.885738e-08,-0.810097,1.614156e-08
2,CN1CC(=O)N(C)CC(=O)N(C)CC(=O)N(C)CC(=O)N(C)CC(...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.842454,-0.969284,0.843293,0.542591,0.659271,-1.028246,-0.227193,0.548413,...,0.003105,0.004603,0.001585,0.016543,-3.390793e-08,0.000710,8.183788e-08,-6.645748e-08,-0.812063,1.538040e-08
3,CN1CCCC(=O)N(C)CC(=O)N(C)CC(=O)N(C)CC(=O)N(C)C...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.244749,-0.718715,0.816326,0.622103,0.842643,-1.134638,0.343202,0.456895,...,0.004024,0.006619,0.001747,0.024863,1.450126e-08,0.000716,9.987627e-08,-7.902280e-08,-0.791207,2.085347e-08
4,CCCN1CC(=O)N(C)CC(=O)N(C)CC(=O)N(C)CC(=O)N(C)C...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.545357,-1.096163,0.435487,1.288385,0.482219,-0.659059,-0.329151,0.365688,...,0.006250,0.008765,0.002352,0.025246,-6.527544e-08,0.000772,1.072502e-07,-8.500248e-08,-0.792563,2.224770e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5210,COCCOCCCCCCCCCCC[Si](OC)(OC)OC,[Li+].O=C1O[B-](F)(F)OC1=O,-0.908149,-0.944494,0.971822,0.043047,-0.735702,0.985443,0.804811,0.287560,...,0.005173,0.004266,0.001007,0.012200,4.994765e-07,0.000670,9.327019e-08,-7.472781e-08,-0.801197,1.854238e-08
5211,CO[Si](OC)(OC)C([Si](OC)(OC)OC)([Si](OC)(OC)OC...,[Li+].O=C1O[B-](F)(F)OC1=O,-0.473794,-1.393888,1.066949,-0.607220,0.353716,-0.159430,0.584239,-0.017302,...,0.004018,0.004784,0.001480,0.018112,2.234199e-07,0.000673,9.419367e-08,-7.520054e-08,-0.798361,1.899313e-08
5212,CCCCCCCCCCCCOCCOCCC[Si](OC)(OC)OC,[Li+].O=C1O[B-](F)(F)OC1=O,-1.076872,-1.681724,0.426816,1.007279,-0.740516,1.312715,0.261993,-0.277779,...,0.007005,0.006652,0.001687,0.015505,3.261720e-07,0.000694,1.059952e-07,-8.418175e-08,-0.794204,2.181342e-08
5213,COCCOCCCOCCCCCCCCCCC[Si](OC)(OC)OC,[Li+].O=C1O[B-](F)(F)OC1=O,-0.915870,-0.938476,0.970829,0.044332,-0.738988,0.996804,0.802128,0.293352,...,0.007381,0.006416,0.001314,0.018434,4.505276e-07,0.000691,1.135464e-07,-8.878854e-08,-0.781958,2.475790e-08


### Check how many common solvents in fourth batch of labeled dataset

In [ ]:
df_label_all = pd.read_csv('../../datasets/label_all_ecfp_pca_add_feat_incl_b7_090824.csv')
df_label_all

,solv_comb_sm,salt_comb_sm,batch,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,...,norm_capacity_15,norm_capacity_16,norm_capacity_17,norm_capacity_18,norm_capacity_19,norm_capacity_20,norm_capacity_21,norm_capacity_22,norm_capacity_23,expt_test
0,COCCOC,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],0.0,-0.845301,-0.995151,1.062720,-0.357552,-0.308720,0.309456,0.693325,...,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,COCCOC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],0.0,-1.183255,-0.948338,0.615257,-0.582269,-1.010187,-0.522075,0.496896,...,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0,0.0
2,COCCOCC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],0.0,-1.240814,-0.970769,0.671105,-0.607789,-1.124651,-0.436617,0.628824,...,0.02168,0.022967,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0,0.0
3,COCCOCC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],0.0,-1.240814,-0.970769,0.671105,-0.607789,-1.124651,-0.436617,0.628824,...,0.00046,0.000360,0.000407,0.0006,0.000593,0.000447,0.000333,0.0,0.0,0.0
4,CCOCCOC(C)(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],0.0,-0.879184,-1.539457,0.507075,-0.111830,0.800175,0.133053,0.926130,...,0.00000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,CCOCCCCOC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0
204,COCCCCOCC(F)(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0
205,COCCCOCC(C)(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0
206,COCCCOCC(F)(F)C(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0


In [31]:
df_b4 = df_label_all.loc[df_label_all['expt_test'] == 4]
df_b4['solv_comb_sm'] = df_b4['solv_comb_sm'].apply(lambda x: Chem.MolToSmiles(Chem.MolFromSmiles(x)))
df_b4

/var/folders/s5/lmr5dh0n4bsgnpxrzwlts5lw0000gn/T/ipykernel_33581/2794166917.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_b4['solv_comb_sm'] = df_b4['solv_comb_sm'].apply(lambda x: Chem.MolToSmiles(Chem.MolFromSmiles(x)))


,solv_comb_sm,salt_comb_sm,batch,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,...,norm_capacity_15,norm_capacity_16,norm_capacity_17,norm_capacity_18,norm_capacity_19,norm_capacity_20,norm_capacity_21,norm_capacity_22,norm_capacity_23,expt_test
131,CCCCCCOC=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,4.0,-1.151047,-1.634688,0.371758,1.375369,0.072907,0.806029,-0.248371,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
132,CCCCCCOC=O,[Li+].F[P-](F)(F)(F)(F)F,4.0,-1.151047,-1.634688,0.371758,1.375369,0.072907,0.806029,-0.248371,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
133,CCCCCCOC=O,[Li+].O=C1O[B-](F)(F)OC1=O,4.0,-1.151047,-1.634688,0.371758,1.375369,0.072907,0.806029,-0.248371,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
134,CCCCCCOS(C)(=O)=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,4.0,-1.185309,-1.639031,0.285152,1.403113,0.107494,0.794562,-0.189131,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
135,CCCCCCOS(C)(=O)=O,[Li+].F[P-](F)(F)(F)(F)F,4.0,-1.185309,-1.639031,0.285152,1.403113,0.107494,0.794562,-0.189131,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
136,CCCCCCOS(C)(=O)=O,[Li+].O=C1O[B-](F)(F)OC1=O,4.0,-1.185309,-1.639031,0.285152,1.403113,0.107494,0.794562,-0.189131,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
137,CCCCCOCCCCC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,4.0,-0.787099,-1.860123,0.108463,1.068635,-0.232016,1.043709,0.085860,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
138,CCCCOCCCl,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,4.0,-0.744628,-1.849104,0.136047,0.898735,-0.158824,0.886538,0.120523,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
139,CCCCOCCCl,[Li+].F[P-](F)(F)(F)(F)F,4.0,-0.744628,-1.849104,0.136047,0.898735,-0.158824,0.886538,0.120523,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
140,CCCCOCCCl,[Li+].O=C1O[B-](F)(F)OC1=O,4.0,-0.744628,-1.849104,0.136047,0.898735,-0.158824,0.886538,0.120523,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0


In [32]:
uniq_sm_b4 = df_b4['solv_comb_sm'].unique()
uniq_sm_b4 = pd.DataFrame(uniq_sm_b4, columns=['solv_comb_sm'])
uniq_sm_b4

,solv_comb_sm
0,CCCCCCOC=O
1,CCCCCCOS(C)(=O)=O
2,CCCCCOCCCCC
3,CCCCOCCCl
4,CCCCOCCOCCCC
5,CCCCOC
6,CCCCCCON=O
7,CO[Si](CCC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(...
8,CCCCC(OCC)OCC


In [33]:
df_comm = df_unlabel_uniq.merge(uniq_sm_b4, on='solv_comb_sm', how='right')
df_comm.dropna(inplace=True)
df_comm

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,EI_1,EI_2,EI_3,EI_4,prediction_aggr,uncertainty_aggr,explore_aggr,exploit_aggr,ratio_aggr,EI_aggr
0,CCCCCCOC=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.151047,-1.634688,0.371758,1.375369,0.072907,0.806029,-0.248371,-0.164916,...,0.005714,0.003043,0.003050,0.001584,1.346934e-06,0.000803,8.293238e-08,-6.766799e-08,-0.815942,1.526439e-08
1,CCCCCCOS(C)(=O)=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.185309,-1.639031,0.285152,1.403113,0.107494,0.794562,-0.189131,-0.101360,...,0.006537,0.003117,0.002408,0.001481,1.329733e-06,0.000798,8.228728e-08,-6.688179e-08,-0.812784,1.540549e-08
2,CCCCCOCCCCC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.787099,-1.860123,0.108463,1.068635,-0.232016,1.043709,0.085860,-0.335381,...,0.005911,0.002879,0.003207,0.001077,1.398512e-06,0.000812,8.074000e-08,-6.576447e-08,-0.814522,1.497553e-08
3,CCCCOCCCl,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.744628,-1.849104,0.136047,0.898735,-0.158824,0.886538,0.120523,-0.324358,...,0.005195,0.002830,0.003967,0.001161,1.556429e-06,0.000854,8.364906e-08,-6.813709e-08,-0.814559,1.551197e-08
5,CCCCOC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.185937,-1.761041,0.562131,0.697612,-0.655805,1.009843,0.605023,0.421360,...,0.010018,0.005251,0.002780,0.001854,1.803637e-06,0.000936,1.106345e-07,-8.667782e-08,-0.783461,2.395670e-08
7,CO[Si](CCC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.703865,-1.378061,0.988412,-0.661167,0.540599,-0.224957,0.849146,-0.022704,...,0.009972,0.004389,0.001291,0.004390,8.291925e-07,0.000712,9.585460e-08,-7.617448e-08,-0.794688,1.968013e-08


In [ ]:
df_comm.to_csv('../../datasets/batch-4/labeled_batch4_uniq_solvents_wo_nmc_data.csv', index=False)

In [43]:
df_comm_label = df_unlabel_uniq.merge(df_b4, on=['solv_comb_sm'], how='left', suffixes=('', '_drop'))
df_comm_label.dropna(inplace=True)
df_comm_label = df_comm_label.loc[:, ~df_comm_label.columns.str.endswith('_drop')]
df_comm_label

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,norm_capacity_15,norm_capacity_16,norm_capacity_17,norm_capacity_18,norm_capacity_19,norm_capacity_20,norm_capacity_21,norm_capacity_22,norm_capacity_23,expt_test
1088,CCCCOC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.185937,-1.761041,0.562131,0.697612,-0.655805,1.009843,0.605023,0.421360,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2586,CO[Si](CCC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.703865,-1.378061,0.988412,-0.661167,0.540599,-0.224957,0.849146,-0.022704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2587,CO[Si](CCC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.703865,-1.378061,0.988412,-0.661167,0.540599,-0.224957,0.849146,-0.022704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2588,CO[Si](CCC(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(...,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.703865,-1.378061,0.988412,-0.661167,0.540599,-0.224957,0.849146,-0.022704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
6134,CCCCOCCCl,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.744628,-1.849104,0.136047,0.898735,-0.158824,0.886538,0.120523,-0.324358,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
6135,CCCCOCCCl,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.744628,-1.849104,0.136047,0.898735,-0.158824,0.886538,0.120523,-0.324358,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
6136,CCCCOCCCl,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.744628,-1.849104,0.136047,0.898735,-0.158824,0.886538,0.120523,-0.324358,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
6282,CCCCCCOS(C)(=O)=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.185309,-1.639031,0.285152,1.403113,0.107494,0.794562,-0.189131,-0.101360,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
6283,CCCCCCOS(C)(=O)=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.185309,-1.639031,0.285152,1.403113,0.107494,0.794562,-0.189131,-0.101360,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
6284,CCCCCCOS(C)(=O)=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.185309,-1.639031,0.285152,1.403113,0.107494,0.794562,-0.189131,-0.101360,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0


In [ ]:
df_comm_label.to_csv('../../datasets/batch-4/labeled_batch4_all_wo_nmc_data.csv', index=False)

In [35]:
df_uncomm = df_unlabel_uniq.merge(uniq_sm_b4, on='solv_comb_sm', how='left')
df_uncomm

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,EI_1,EI_2,EI_3,EI_4,prediction_aggr,uncertainty_aggr,explore_aggr,exploit_aggr,ratio_aggr,EI_aggr
0,COCCOCCOC(F)(F)Cl,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.013246,-1.132293,1.203856,-0.549821,0.238278,0.290024,1.338129,0.446974,...,0.008311,0.010022,0.009974,0.004824,3.927265e-06,0.001535,2.032648e-07,-1.421704e-07,-0.699434,6.109439e-08
1,COCCOCC(F)(OC)C(F)(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.020865,-1.124835,1.256503,-0.548366,0.183966,0.306774,1.383833,0.462789,...,0.008990,0.010922,0.009346,0.005423,3.647808e-06,0.001467,2.036904e-07,-1.445650e-07,-0.709729,5.912541e-08
2,COCCOCCC(F)(F)C(F)(F)F,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.995461,-1.139384,1.208259,-0.557954,0.205390,0.322203,1.391092,0.433611,...,0.008560,0.010429,0.010905,0.005210,3.500622e-06,0.001431,2.022702e-07,-1.431818e-07,-0.707874,5.908839e-08
3,COCCOCC(F)(F)OCF,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.979777,-1.126940,1.238547,-0.543210,0.172293,0.267553,1.378745,0.361646,...,0.008655,0.010482,0.012502,0.005346,3.262107e-06,0.001371,2.025798e-07,-1.438307e-07,-0.709995,5.874905e-08
4,COCCOCCOCCOCCOCC(F)(F)CCCCC[Si](OC)(OC)OC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.309362,-1.134852,1.181017,-0.133862,0.039111,0.605024,1.403487,0.326658,...,0.027248,0.013353,0.001753,0.017335,1.273534e-06,0.000879,1.882993e-07,-1.301604e-07,-0.691242,5.813893e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7961,CCCOOC,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.920924,-1.603036,0.550274,0.249128,-0.250719,0.486272,0.282188,0.202565,...,0.003560,0.001788,0.005234,0.000239,1.892553e-06,0.000933,7.407750e-08,-5.979962e-08,-0.807257,1.427788e-08
7962,COCCC(=O)N1CCN(C(=O)C(C)(C)C)CC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.281381,0.522806,1.360425,0.402163,0.443373,-0.153257,1.562089,0.615370,...,0.006078,0.003086,0.000954,0.004148,7.580584e-07,0.000681,7.792661e-08,-6.365001e-08,-0.816794,1.427660e-08
7963,COC(=O)CC1C(=O)N(C)CCN1C(=O)N1CCC(C)(OC)C1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.904417,0.914748,1.975455,0.537320,-0.080118,-0.456326,-0.384179,1.347751,...,0.005910,0.003914,0.001474,0.008205,1.450842e-07,0.000674,7.929405e-08,-6.501750e-08,-0.819954,1.427654e-08
7964,COCCOCCOCCCC(=O)C(C)(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-1.051868,-0.115286,1.282676,-0.037341,0.381647,0.783399,1.163550,0.014020,...,0.006641,0.003090,0.000989,0.001506,1.299337e-06,0.000789,7.584306e-08,-6.156749e-08,-0.811775,1.427557e-08


In [36]:
uniq_sm_comm = df_comm['solv_comb_sm'].unique()
for i in range(len(uniq_sm_b4)):
    if uniq_sm_b4['solv_comb_sm'][i] not in uniq_sm_comm:
        print(uniq_sm_b4['solv_comb_sm'][i])

CCCCOCCOCCCC
CCCCCCON=O
CCCCC(OCC)OCC
